In [5]:
#Note I would like to do a reAct thing here because it is cool but ...
#I ain't writing those massive few shot examples for nothing but pratice I think I get the gist lol
#pip install python-dotenv google-generativeai enum typing
import google.generativeai as genai
from google.api_core import retry
from dotenv import load_dotenv
import os
import enum
import typing_extensions

In [ ]:
load_dotenv()

class EnumOfGood(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"
class JSONResponse(typing_extensions.TypedDict):
    Sentiment: EnumOfGood
    why: str

GoogleApiKey = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GoogleApiKey)
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}

configuration = genai.GenerationConfig(
    temperature=.1, #how "random" the model is (how much is deviates from top choices)
    top_k=2, #the top k choices from the prompt
    top_p=.9, #once the reponses reach a cumulative 90% stop
    response_schema=JSONResponse
)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=configuration)

In [26]:
one_shot_prompt = ("Rate the sentiment of the statement and classify it as positive, neutral, or negative "
                   "STATEMENT: The pizza was really good!"
                   "{Sentiment: positive,"
                   "why: the person claims the pizza was really good expressing positive excitement toward something}"
                   "STATEMENT: I can't believe you would do that!")
response = model.generate_content(one_shot_prompt, request_options=retry_policy)
print(response.text)

Sentiment: Negative

Why:  The statement expresses disbelief and strong disapproval, indicating a negative emotional response.  The phrase "I can't believe" often precedes a negative judgment or criticism.

